In [ ]:
from flask import Flask, request, jsonify
from kafka import KafkaProducer
import pandas as pd
import json
import os

# Load data from CSV file
data_file = "C:/Users/hughp/Downloads/cleaned_data.csv"
if not os.path.exists(data_file):
    raise FileNotFoundError(f"Data file not found at path: {data_file}")

df = pd.read_csv(data_file)

# Set up Kafka producer
KAFKA_BROKER = "localhost:9092"  # Replace with your Kafka broker
TOPIC_NAME = "csv_data_topic"
producer = KafkaProducer(
    bootstrap_servers=[KAFKA_BROKER],
    value_serializer=lambda x: json.dumps(x).encode("utf-8")
)

# Set up Flask app
app = Flask(__name__)

@app.route('/send-data', methods=['POST'])
def send_data():
    start = int(request.args.get("start", 0))
    end = int(request.args.get("end", len(df)))

    if start >= end or start < 0 or end > len(df):
        return jsonify({"error": "Invalid start or end index"}), 400

    # Send rows from DataFrame as messages to Kafka
    for _, row in df.iloc[start:end].iterrows():
        producer.send(TOPIC_NAME, row.to_dict())
    producer.flush()
    return jsonify({"message": f"Data sent from row {start} to {end}"}), 200

@app.route('/')
def index():
    return "Kafka Producer API is running."

if __name__ == "__main__":
    app.run(port=5000)
